In [1]:
#tweet cleaning
#https://www.kaggle.com/ragnisah/text-data-cleaning-tweets-analysis
#https://towardsdatascience.com/basic-data-cleaning-engineering-session-twitter-sentiment-data-95e5bd2869ec

#Topic Modeilling
#https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

#Cleaning
#Puncuation
#lower case
#stopwords
#abbreviations

# Importing modules
import pandas as pd
import os

# Load the regular expression library
import re

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk as nltk
nltk.download('stopwords')
nltk.download('wordnet')
import gensim
from gensim.utils import simple_preprocess
from gensim.models.wrappers import LdaMallet
from nltk.corpus import stopwords
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import pyLDAvis.gensim
import pickle 
import pyLDAvis
#import spacy
#from scispacy.abbreviation import AbbreviationDetector

#sentiment analysis
#https://medium.com/swlh/simple-sentiment-analysis-for-nlp-beginners-and-everyone-else-using-vader-and-textblob-728da3dbe33d
import sqlite3
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\megha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\megha\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [2]:
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\megha\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
diabetes2019 = pd.read_csv('diabetes2019_lda.csv')

In [4]:
# Print head
diabetes2019.head(10)

,Post ID,Author,Publish Date,Body,text_gens_nonstop,text_processed,text_tokenized,text_nlk_nonstop,text_nonstop_spacy,text_stemmed,text_lemmatized,Dominant,month_year
0,abi6me,buttersmacks,2019-01-01 05:27:00,"Keto.\nStressed.\nDiabetic (cannot risk any higher sugars)\nCdiff, crohns.\n\nDoc wants to dry L...","Keto. Stressed. Diabetic (cannot risk higher sugars) Cdiff, crohns. Doc wants dry LDN. Anyone tr...",keto stressed diabetic cannot risk higher sugars cdiff crohns doc wants dry ldn anyone try issue...,"['keto', 'stressed', 'diabetic', 'cannot', 'risk', 'higher', 'sugars', 'cdiff', 'crohns', 'doc',...","['keto', 'stressed', 'diabetic', 'cannot', 'risk', 'higher', 'sugars', 'cdiff', 'crohns', 'doc',...","['keto', 'stressed', 'diabetic', 'risk', 'higher', 'sugars', 'cdiff', 'crohns', 'doc', 'wants', ...","['keto', 'stress', 'diabet', 'risk', 'higher', 'sugar', 'cdiff', 'crohn', 'doc', 'want', 'dri', ...","['keto', 'stressed', 'diabetic', 'risk', 'higher', 'sugar', 'cdiff', 'crohn', 'doc', 'want', 'dr...",Topic 7,2019-01
1,ed3g2mf,SusanW100,2019-01-02 11:15:00,I have manged to bring my blood sugar levels to normal range after being all the time over 200's...,I manged bring blood sugar levels normal range time 200's. And I managed 3 weeks. I diabetic yea...,i manged bring blood sugar levels normal range time s and i managed weeks i diabetic years trie...,"['i', 'manged', 'bring', 'blood', 'sugar', 'levels', 'normal', 'range', 'time', 's', 'and', 'i',...","['manged', 'bring', 'blood', 'sugar', 'levels', 'normal', 'range', 'time', 'managed', 'weeks', '...","['manged', 'bring', 'blood', 'sugar', 'levels', 'normal', 'range', 'time', 'managed', 'weeks', '...","['mang', 'bring', 'blood', 'sugar', 'level', 'normal', 'rang', 'time', 'manag', 'week', 'diabet'...","['manged', 'bring', 'blood', 'sugar', 'level', 'normal', 'range', 'time', 'managed', 'week', 'di...",Topic 6,2019-01
2,ed3h251,MiguelTall,2019-01-02 11:27:00,"Thanks to Susan here on the board, I have gone on the libre blood sugar program. It's a blessing...","Thanks Susan board, I gone libre blood sugar program. It's blessing I program body start deterio...",thanks susan board i gone libre blood sugar program its blessing i program body start deteriote ...,"['thanks', 'susan', 'board', 'i', 'gone', 'libre', 'blood', 'sugar', 'program', 'its', 'blessing...","['thanks', 'susan', 'board', 'gone', 'libre', 'blood', 'sugar', 'program', 'blessing', 'program'...","['thanks', 'susan', 'board', 'gone', 'libre', 'blood', 'sugar', 'program', 'blessing', 'program'...","['thank', 'susan', 'board', 'gone', 'libr', 'blood', 'sugar', 'program', 'bless', 'program', 'bo...","['thanks', 'susan', 'board', 'gone', 'libre', 'blood', 'sugar', 'program', 'blessing', 'program'...",Topic 9,2019-01
3,ed2qyr6,ketocultmember,2019-01-02 05:17:00,"This big spammy ad isn’t telling anything, either, but it will see the “info.”","This big spammy ad isn’t telling anything, either, “info.”",this big spammy ad isn’t telling anything either “info”,"['this', 'big', 'spammy', 'ad', 'isn', 't', 'telling', 'anything', 'either', 'info', '']","['big', 'spammy', 'ad', 'telling', 'anything', 'either', 'info', '']","['big', 'spammy', 'ad', 'telling', 'info', '']","['big', 'spammi', 'ad', 'tell', 'info', '']","['big', 'spammy', 'ad', 'telling', 'info', '']",Topic 8,2019-01
4,ed2zhvk,adawah,2019-01-02 07:44:00,That looks about like how my days go. Anytime I can see those 100's makes for a good day.,That looks like days go. Anytime I 100's makes good day.,that looks like days go anytime i s makes good day,"['that', 'looks', 'like', 'days', 'go', 'anytime', 'i', 's', 'makes', 'good', 'day']","['looks', 'days', 'go', 'anytime', 'makes', 'good', 'day']","['looks', 'days', 'anytime', 'makes', 'good', 'day']","['look', 'day', 'anytim', 'make', 'good', 'day']","['look', 'day', 'anytime', 'make', 'good', 'day']",Topic 10,2019-01
5,ed34ob4,BelladonnaBunny,2019-01-02 08:54:00,Right! My timing has been off on my meal insulin lately 

In [5]:
list(diabetes2019.columns)

['Post ID',
 'Author',
 'Publish Date',
 'Body',
 'text_gens_nonstop',
 'text_processed',
 'text_tokenized',
 'text_nlk_nonstop',
 'text_nonstop_spacy',
 'text_stemmed',
 'text_lemmatized',
 'Dominant',
 'month_year']

In [6]:
#drop unnecessary columns
diabetes2019 = diabetes2019.drop(columns = ['text_gens_nonstop', 'text_processed', 'text_tokenized', 'text_nlk_nonstop', 'text_nonstop_spacy', 'text_stemmed', 'text_lemmatized'])
diabetes2019

,Post ID,Author,Publish Date,Body,Dominant,month_year
0,abi6me,buttersmacks,2019-01-01 05:27:00,"Keto.\nStressed.\nDiabetic (cannot risk any higher sugars)\nCdiff, crohns.\n\nDoc wants to dry L...",Topic 7,2019-01
1,ed3g2mf,SusanW100,2019-01-02 11:15:00,I have manged to bring my blood sugar levels to normal range after being all the time over 200's...,Topic 6,2019-01
2,ed3h251,MiguelTall,2019-01-02 11:27:00,"Thanks to Susan here on the board, I have gone on the libre blood sugar program. It's a blessing...",Topic 9,2019-01
3,ed2qyr6,ketocultmember,2019-01-02 05:17:00,"This big spammy ad isn’t telling anything, either, but it will see the “info.”",Topic 8,2019-01
4,ed2zhvk,adawah,2019-01-02 07:44:00,That looks about like how my days go. Anytime I can see those 100's makes for a good day.,Topic 10,2019-01
...,...,...,...,...,...,...
28772,fcn9uqy,Droseph13,2019-12-31 07:42:00,"Both I cant sleep when my sugar is too high, and lack of sleep causes my numbers to be higher th...",Topic 3,2019-12
28773,fcpfmn1,CursedEarthPizza,2019-12-31 22:57:00,I think it starts with high sugar levels. I was slamming multiple fountain drinks and cans of c...,Topic 3,2019-12
28774,fcswfr0,forst1tj,2020-01-01 17:00:00,My Dr. put me on Rybelsus about 2 mo ago. It’s basically a pill version of Ozempic..the side ef...,Topic 9,2020-01
28775,fcwscpl,alan_s,2020-01-02 23:14:00,The sleep problem is probably related to your high levels but those are more likely related to y...,Topic 7,2020-01


In [7]:
types = diabetes2019.dtypes
types

Post ID         object
Author          object
Publish Date    object
Body            object
Dominant        object
month_year      object
dtype: object

In [9]:
#convert object to string
diabetes2019['Body'] = diabetes2019['Body'].astype(str)

In [11]:
#sentiment analysis with textblob
#https://medium.com/swlh/simple-sentiment-analysis-for-nlp-beginners-and-everyone-else-using-vader-and-textblob-728da3dbe33d

#load the descriptions into textblob
desc_blob = [TextBlob(desc) for desc in diabetes2019['Body']]

#add the sentiment metrics to the dataframe
diabetes2019['tb_Pol'] = [b.sentiment.polarity for b in desc_blob]
diabetes2019['tb_Subj'] = [b.sentiment.subjectivity for b in desc_blob]

diabetes2019.head()

,Post ID,Author,Publish Date,Body,Dominant,month_year,tb_Pol,tb_Subj
0,abi6me,buttersmacks,2019-01-01 05:27:00,"Keto.\nStressed.\nDiabetic (cannot risk any higher sugars)\nCdiff, crohns.\n\nDoc wants to dry L...",Topic 7,2019-01,0.127778,0.400000
1,ed3g2mf,SusanW100,2019-01-02 11:15:00,I have manged to bring my blood sugar levels to normal range after being all the time over 200's...,Topic 6,2019-01,0.290404,0.639899
2,ed3h251,MiguelTall,2019-01-02 11:27:00,"Thanks to Susan here on the board, I have gone on the libre blood sugar program. It's a blessing...",Topic 9,2019-01,0.090139,0.393611
3,ed2qyr6,ketocultmember,2019-01-02 05:17:00,"This big spammy ad isn’t telling anything, either, but it will see the “info.”",Topic 8,2019-01,0.000000,0.100000
4,ed2zhvk,adawah,2019-01-02 07:44:00,That looks about like how my days go. Anytime I can see those 100's makes for a good day.,Topic 10,2019-01,0.700000,0.600000


In [14]:
#load VADER
analyzer = SentimentIntensityAnalyzer()

#Add VADER metrics to dataframe
diabetes2019['compound'] = [analyzer.polarity_scores(v)['compound'] for v in diabetes2019['Body']]
diabetes2019['neg'] = [analyzer.polarity_scores(v)['neg'] for v in diabetes2019['Body']]
diabetes2019['neu'] = [analyzer.polarity_scores(v)['neu'] for v in diabetes2019['Body']]
diabetes2019['pos'] = [analyzer.polarity_scores(v)['pos'] for v in diabetes2019['Body']]

diabetes2019.tail(50)

,Post ID,Author,Publish Date,Body,Dominant,month_year,tb_Pol,tb_Subj,compound,neg,neu,pos
28727,fcavk1l,LetsGoHawks,2019-12-28 15:57:00,You keep responding with a copy/paste to every post. What are you trying to get at here?,Topic 7,2019-12,0.000000,0.000000,0.0000,0.000,1.000,0.000
28728,fcawhc4,goodfornothingwala,2019-12-28 16:06:00,I want to know about their cases how do I reply with a comment that everyone sees?,Topic 7,2019-12,0.000000,0.000000,0.0772,0.000,0.925,0.075
28729,fcaxn5k,LetsGoHawks,2019-12-28 16:17:00,"Everybody sees every comment. \n\nSo, again, what is it you're trying to get at? What are you tr...",Topic 7,2019-12,0.000000,0.000000,0.0000,0.000,1.000,0.000
28730,fcb0al6,Starryglare,2019-12-28 16:42:00,"I had an ultrasound that didn't see any fatty liver. in my case, the resistance is due to hormon...",Topic 6,2019-12,-0.162500,0.387500,-0.5719,0.150,0.850,0.000
28731,fcb34vh,goodfornothingwala,2019-12-28 17:08:00,"Really, I don't get notifications about threads not involving me. My housemate has been fasting ...",Topic 1,2019-12,0.000000,0.300000,0.0000,0.000,1.000,0.000
28732,fcb4u1f,LetsGoHawks,2019-12-28 17:19:00,"As it pertains to Type 2 Diabetes, ""reversal"" just means ""well controlled"", not cured. T2D is, u...",Topic 8,2019-12,0.380000,0.780000,0.9015,0.026,0.767,0.207
28733,fcb5t3x,LetsGoHawks,2019-12-28 17:25:00,"Genetics is the biggest factor. \n\nI'm 5'11"", spent most of my life under 200lb. Never weighed ...",Topic 4,2019-12,0.160582,0.390476,-0.6486,0.095,0.862,0.043
28734,fcdk4d1,FlowMang,2019-12-29 01:12:00,IMO fatty liver and obesity are symptoms of T2. Many people seem to think it’s the other way aro...,Topic 6,2019-12,0.058333,0.425000,0.6486,0.000,0.880,0.120
28735,fcjmhvq,SplatterQuillon,2019-12-30 00:18:00,"A liver emzyme test won't confirm absence of fatty liver. Your liver can be extremely fatty,. ...",Topic 1,2019-12,-0.004688,0.384375,0.6124,0.000,0.922,0.078
28736,fcjmqz3,alan_s,2019-12-30 00:23:00,I have been diagnosed with type 2 diabetes for seventeen years. I have never been diagnosed with...,Topic 8,2019-12,-0.225000,0.450000,0.0000,0.000,1.000,0.000


In [15]:
#drop textblob columns
diabetes2019 = diabetes2019.drop(columns = ['tb_Pol', 'tb_Subj'])
diabetes2019

,Post ID,Author,Publish Date,Body,Dominant,month_year,compound,neg,neu,pos
0,abi6me,buttersmacks,2019-01-01 05:27:00,"Keto.\nStressed.\nDiabetic (cannot risk any higher sugars)\nCdiff, crohns.\n\nDoc wants to dry L...",Topic 7,2019-01,-0.1496,0.085,0.851,0.064
1,ed3g2mf,SusanW100,2019-01-02 11:15:00,I have manged to bring my blood sugar levels to normal range after being all the time over 200's...,Topic 6,2019-01,0.8852,0.031,0.854,0.115
2,ed3h251,MiguelTall,2019-01-02 11:27:00,"Thanks to Susan here on the board, I have gone on the libre blood sugar program. It's a blessing...",Topic 9,2019-01,0.9286,0.018,0.846,0.136
3,ed2qyr6,ketocultmember,2019-01-02 05:17:00,"This big spammy ad isn’t telling anything, either, but it will see the “info.”",Topic 8,2019-01,0.0000,0.000,1.000,0.000
4,ed2zhvk,adawah,2019-01-02 07:44:00,That looks about like how my days go. Anytime I can see those 100's makes for a good day.,Topic 10,2019-01,0.6597,0.000,0.759,0.241
...,...,...,...,...,...,...,...,...,...,...
28772,fcn9uqy,Droseph13,2019-12-31 07:42:00,"Both I cant sleep when my sugar is too high, and lack of sleep causes my numbers to be higher th...",Topic 3,2019-12,0.3400,0.099,0.785,0.116
28773,fcpfmn1,CursedEarthPizza,2019-12-31 22:57:00,I think it starts with high sugar levels. I was slamming multiple fountain drinks and cans of c...,Topic 3,2019-12,-0.1531,0.049,0.920,0.031
28774,fcswfr0,forst1tj,2020-01-01 17:00:00,My Dr. put me on Rybelsus about 2 mo ago. It’s basically a pill version of Ozempic..the side ef...,Topic 9,2020-01,0.2263,0.000,0.936,0.064
28775,fcwscpl,alan_s,2020-01-02 23:14:00,The sleep problem is probably related to your high levels but those are more likely related to y...,Topic 7,2020-01,-0.8516,0.102,0.898,0.000


In [16]:
#download data with topics and sentiment score
diabetes2019.to_csv('diabetes2019_sentiment.csv', index = False)

In [17]:
#group average sentiment by topic
average_sentiment = diabetes2019.groupby('Dominant')['compound', 'neg', 'neu', 'pos'].mean()
average_sentiment

<ipython-input-17-776db7532b72>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_sentiment = diabetes2019.groupby('Dominant')['compound', 'neg', 'neu', 'pos'].mean()


,compound,neg,neu,pos
Dominant,,,,
Topic 1,0.192033,0.043042,0.832998,0.123671
Topic 10,0.482147,0.063479,0.644557,0.291964
Topic 2,0.147322,0.063856,0.846681,0.089457
Topic 3,0.096073,0.066455,0.839885,0.093668
Topic 4,0.220276,0.075389,0.796082,0.128512
Topic 5,0.271409,0.055659,0.828327,0.116009
Topic 6,0.084623,0.069766,0.843318,0.086915
Topic 7,0.166865,0.082723,0.753463,0.163825
Topic 8,0.179137,0.072170,0.807741,0.120088


In [18]:
average_sentiment.to_csv('diabetes2019_sentiment_by_topic.csv', index = False)

In [19]:
#group by month 
average_sentiment_month = diabetes2019.groupby('month_year')['compound', 'neg', 'neu', 'pos'].mean()
average_sentiment_month

<ipython-input-19-b71649c4fcad>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_sentiment_month = diabetes2019.groupby('month_year')['compound', 'neg', 'neu', 'pos'].mean()


,compound,neg,neu,pos
month_year,,,,
2019-01,0.227691,0.060910,0.801561,0.137513
2019-02,0.234120,0.065984,0.801639,0.132381
2019-03,0.224021,0.062524,0.804578,0.132899
2019-04,0.210733,0.060496,0.807230,0.131818
2019-05,0.230294,0.062728,0.800790,0.136469
2019-06,0.230360,0.058899,0.802663,0.138432
2019-07,0.212445,0.064349,0.795842,0.139809
2019-08,0.207504,0.061842,0.801717,0.136446
2019-09,0.182357,0.068053,0.808230,0.123711


In [20]:
average_sentiment_month.to_csv('diabetes2019_sentiment_by_month.csv', index = False)

In [21]:
#sort by authors & dates
diabetes_authors = diabetes2019.sort_values(by=['Author', 'Publish Date'])
diabetes_authors

,Post ID,Author,Publish Date,Body,Dominant,month_year,compound,neg,neu,pos
17484,ejs9c61,#NAME?,2019-03-30 21:05:00,"I've been on it only for two weeks, but have had no adverse effects so far. My blood sugar has b...",Topic 6,2019-03,0.7694,0.065,0.804,0.131
19780,emx2713,#NAME?,2019-05-09 06:21:00,I'm allergic to skin-tac so I just use medical tape. Got like 200 meters of it on amazon for $10,Topic 9,2019-05,0.2500,0.094,0.726,0.179
22095,ey2jou5,--DQ--,2019-08-25 06:54:00,"For me this has been the most dangerous aspect of diabetes, worse than DKA. Fortunately I have n...",Topic 9,2019-08,-0.6729,0.078,0.885,0.038
22188,eynmbam,--DQ--,2019-08-31 04:57:00,Keep in mind that that meter is probably accurate to +/- 20% when above 70 and +/- 20 mg/dl when...,Topic 1,2019-08,-0.3919,0.048,0.952,0.000
22406,ezjtn7r,--DQ--,2019-09-08 06:43:00,"Margin of error for most meters is +/- 20%, so If you want to be 125 +/- 10 that is 92-162 on th...",Topic 9,2019-09,-0.2815,0.086,0.864,0.050
...,...,...,...,...,...,...,...,...,...,...
4725,em55grd,zmmm268,2019-04-30 08:07:00,"Thanks everyone for the comments. I took it, had no noticeable side effects. My medical school ...",Topic 3,2019-04,0.5100,0.016,0.925,0.059
4966,blmev6,zmmm268,2019-05-06 21:37:00,My daughter is graduating college and we will be taking her out to dinner. She has informed me s...,Topic 5,2019-05,0.6808,0.000,0.913,0.087
4977,emtkv0r,zmmm268,2019-05-08 04:42:00,I beg to differ. The experiences everyone has expressed tend to mimic my own. I never would hav...,Topic 1,2019-05,0.9616,0.017,0.812,0.170
8323,etwcewd,zmmm268,2019-07-16 00:46:00,"I have tried gin, vodka and tequila all with either flavored seltzer or diet soda. None have cau...",Topic 5,2019-07,-0.2732,0.049,0.951,0.000


In [22]:
diabetes_authors.to_csv('diabetes2019_authors_sentiment.csv', index = False)